# Production-ready with Kornia

This tutorial presents the integration of Kornia image processings and neural networks. In real-world deployment, engineers commonly use OpenCV to read images then perform a bunch of preprocessing protocols to the image, which is sophisticated in C++ and likely to be buggy due to the decoupling of image preprocessing and many trained neural networks. In this sense, it is always a good practice to make the model grab-and-use.

### Integrate neural image processors into PyTorch model

In [ ]:
import kornia
import torch
import torch.nn as nn

model_with_proc = nn.Sequential(
    kornia.color.BgrToRgb(),
    kornia.enhance.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    kornia.contrib.face_detection.YuFaceDetectNet("test", pretrained=True)
)

## Convert model to ONNX 

[ONNX](https://onnx.ai) is an open format built to represent machine learning models. ONNX is the first step in enabling more of these tools to work together by allowing them to share models.

In [ ]:
dummy_input = torch.randn(10, 3, 224, 224)

torch.onnx.export(
    model_with_proc, dummy_input, "yunet_with_proc.onnx", verbose=False,
    input_names=["input"], output_names=["output"], opset_version=13
)

### [Optional] Convert to TensorRT

If you are working with NVIDIA Jetson series, you may want to refer to [trtexec](https://docs.nvidia.com/deeplearning/tensorrt/developer-guide/index.html#trtexec) for the detailed command explaination.

Commonly, you may use

```bash
./trtexec  --onnx=yunet_with_proc.onnx \
           --minShapes=input:1x3x224x224 \
           --optShapes=input:1x3x224x224 \
           --maxShapes=input:1x3x224x224 \
           --saveEngine=alexnet_with_proc_fp16.engine \
           --fp16
```